In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import missingno as msno


In [34]:
from sklearn.preprocessing import StandardScaler

In [341]:
df = pd.read_csv("lending_train.csv"); val = "train"
# df = pd.read_csv("lending_topredict.csv"); val = "test"
df.head()

,ID,requested_amnt,loan_duration,employment,employment_length,race,reason_for_loan,extended_reason,annual_income,debt_to_income_ratio,...,delinquency_last_2yrs,fico_score_range_low,fico_score_range_high,fico_inquired_last_6mths,months_since_last_delinq,revolving_balance,total_revolving_limit,type_of_application,any_tax_liens,loan_paid
0,1000000,9600.0,36 months,Computer Technician,< 1 year,W,debt_consolidation,Debt consolidation,30000.0,19.01,...,0.0,680.0,684.0,0.0,NaN,18492.0,22900.0,Individual,0.0,?
1,1000001,8000.0,36 months,Teacher,6 years,B,debt_consolidation,Debt consolidation,34500.0,31.30,...,0.0,650.0,654.0,0.0,NaN,5119.0,26000.0,Individual,0.0,?
2,1000002,21000.0,36 months,Portability Specialist,10+ years,B,other,Other,45000.0,13.15,...,0.0,650.0,654.0,0.0,NaN,5579.0,8700.0,Individual,0.0,?
3,1000003,5000.0,36 months,Internal Maintence,2 years,B,credit_card,Credit card refinancing,30000.0,6.74,...,0.0,650.0,654.0,1.0,NaN,3872.0,22100.0,Individual,0.0,?
4,1000004,3000.0,36 months,Office administration,< 1 year,B,other,Other,38000.0,0.79,...,0.0,650.0,654.0,0.0,NaN,250.0,8800.0,Individual,0.0,?


In [342]:
df.shape

(345310, 25)

In [343]:
msno.matrix(df)

In [344]:
df.isnull().sum(axis = 0)

In [345]:
# columns to keep 
# requested_amnt, loan_duration, employment_length, reason_for_loan, annual_income (requires rounding), debt_to_income_ratio (normalize)
# employment_verified, public_bankruptcies, home_ownership_status,
# 
# delinquency_last_2yrs, fico_score_range_low, fico_score_range_high, fico_inquired_last_6mths, 
# revolving_balance, any_tax_liens
 

# drop employment because of large variation of unique values, ABLE TO GROUP BY KNOWLEDGE
# drop race because its illegal

# drop extended reason, because large var, ABLE TO GROUP 
# df["extended_reason"] = df["extended_reason"].str.lower()
# print(df["extended_reason"].nunique())
# df["extended_reason"].value_counts()[df["extended_reason"].value_counts() > 1000]


# drop zipcode, state
# drop type of application low variance in data

# drop total revolving limit due to many null columns
# drop months_since_last_delinq since too many nans


# LOAN DURATION AND TYPE OF APPLICATION


In [346]:
df["type_of_application"].value_counts()
df["loan_duration"].value_counts()

 36 months    262561
 60 months     82749
Name: loan_duration, dtype: int64

In [347]:
new_df = pd.DataFrame()

In [348]:
# print(df["requested_amnt"].nunique())
# print(df["requested_amnt"].isnull().sum())
# df[df["requested_amnt"] > 10_000]["requested_amnt"].count()

new_df["requested_amnt"] = df["requested_amnt"]
new_df.loc[df["requested_amnt"] <= 2_500, "requested_amnt"] = "<_2500"
new_df.loc[df["requested_amnt"] > 2_500, "requested_amnt"] = ">_2500"
new_df.loc[df["requested_amnt"] > 5_000, "requested_amnt"] = ">_5000"
new_df.loc[df["requested_amnt"] > 10_000, "requested_amnt"] = ">_10000"
new_df.loc[df["requested_amnt"] > 20_000, "requested_amnt"] = ">_20000"
new_df.loc[df["requested_amnt"] > 30_000, "requested_amnt"] = ">_30000"

new_df["requested_amnt"].value_counts()

# new_df["requested_amnt"] = df["requested_amnt"]
# scaler = StandardScaler()
# new_df["requested_amnt"] = scaler.fit_transform(df["requested_amnt"].values.reshape(-1, 1))


>_10000    127644
>_5000      96016
>_20000     54224
>_2500      36152
>_30000     20669
<_2500      10605
Name: requested_amnt, dtype: int64

In [349]:
# print(df["employment_length"].nunique())
# print(df["employment_length"].isnull().sum())
# df["employment_length"].value_counts()

new_df["employment_length"] = df["employment_length"]

# condition1 = (df["employment_length"] == "< 1 year") | (df["employment_length"] == "1 year") | (df["employment_length"] == "2 years")
# condition2 = (df["employment_length"] == "3 years") | (df["employment_length"] == "4 years") | (df["employment_length"] == "5 years")
# condition3 = (df["employment_length"] == "6 years") | (df["employment_length"] == "7 years") | (df["employment_length"] == "8 years") | (df["employment_length"] == "9 years")


# new_df.loc[condition1, "employment_length"] = "0-2_years"
# new_df.loc[condition2, "employment_length"] = "3-5_years"
# new_df.loc[condition3, "employment_length"] = "6-9_years"

# new_df["employment_length"].value_counts()

In [350]:
new_df["type_of_application"] = df["type_of_application"]
new_df["loan_duration"] = df["loan_duration"]
new_df["state"] = df["state"]

In [351]:
# print(df["reason_for_loan"].nunique())
# print(df["reason_for_loan"].isnull().sum())
# df["reason_for_loan"].value_counts()

new_df["reason_for_loan"] = df["reason_for_loan"]
new_df.loc[(new_df["reason_for_loan"] != "debt_consolidation") & (new_df["reason_for_loan"] != "credit_card"), "reason_for_loan"] = "other"

new_df["reason_for_loan"].value_counts()

debt_consolidation    200352
credit_card            75648
other                  69310
Name: reason_for_loan, dtype: int64

In [352]:
# print(df["annual_income"].nunique())
# print(df["annual_income"].isnull().sum())
# df[df["annual_income"] > 50_000]["annual_income"].count()

# new_df["annual_income"] = pd.Series()
# new_df.loc[df["annual_income"] <= 25_000, "annual_income"] = "<_25000"
# new_df.loc[df["annual_income"] > 25_000, "annual_income"] = ">_25000"
# new_df.loc[df["annual_income"] > 50_000, "annual_income"] = ">_50000"
# new_df.loc[df["annual_income"] > 75_000, "annual_income"] = ">_75000"
# new_df.loc[df["annual_income"] > 100_000, "annual_income"] = ">_100000"
# new_df["annual_income"].value_counts()

new_df["annual_income"] = df["annual_income"]
if val == "train":
    scaler1 = StandardScaler()
    new_df["annual_income"] = scaler1.fit_transform(df["annual_income"].values.reshape(-1, 1))
else:
    new_df["annual_income"] = scaler1.transform(new_df["annual_income"].values.reshape(-1, 1))


In [353]:
new_df["debt_to_income_ratio"] = df["debt_to_income_ratio"]
new_df['debt_to_income_ratio'] = new_df["debt_to_income_ratio"].fillna(new_df["debt_to_income_ratio"].mean())
if val == "train":
    scaler2 = StandardScaler()
    new_df["debt_to_income_ratio"] = scaler2.fit_transform(new_df["debt_to_income_ratio"].values.reshape(-1, 1))
else:
    new_df["debt_to_income_ratio"] = scaler2.transform(new_df["debt_to_income_ratio"].values.reshape(-1, 1))

In [354]:
# print(df["employment_verified"].nunique())
# print(df["employment_verified"].isnull().sum())
# df["employment_verified"].value_counts()

new_df["employment_verified"] = df["employment_verified"]

In [355]:
new_df.head()

,requested_amnt,employment_length,type_of_application,loan_duration,state,reason_for_loan,annual_income,debt_to_income_ratio,employment_verified
0,>_5000,< 1 year,Individual,36 months,IN,debt_consolidation,-0.655125,0.065807,Verified
1,>_5000,6 years,Individual,36 months,CA,debt_consolidation,-0.591423,1.168144,Source Verified
2,>_20000,10+ years,Individual,36 months,FL,other,-0.442785,-0.459799,Verified
3,>_2500,2 years,Individual,36 months,AZ,credit_card,-0.655125,-1.034737,Not Verified
4,>_2500,< 1 year,Individual,36 months,CA,other,-0.541877,-1.568416,Source Verified


In [356]:
# print(df["public_bankruptcies"].nunique())
# print(df["public_bankruptcies"].isnull().sum())
# df["public_bankruptcies"].value_counts()

new_df["public_bankruptcies"] = df["public_bankruptcies"]
new_df['public_bankruptcies'] = new_df["public_bankruptcies"].fillna(0)

new_df.loc[new_df["public_bankruptcies"] == 0, "public_bankruptcies"] = "==0"
new_df.loc[df["public_bankruptcies"] == 1, "public_bankruptcies"] = "==1"
new_df.loc[df["public_bankruptcies"] >= 2, "public_bankruptcies"] = ">=2"

new_df["public_bankruptcies"].value_counts()

# scaler = StandardScaler()
# new_df["public_bankruptcies"] = scaler.fit_transform(new_df["public_bankruptcies"].values.reshape(-1, 1))

==0    302367
==1     40366
>=2      2577
Name: public_bankruptcies, dtype: int64

In [357]:
# print(df["home_ownership_status"].unique())
# print(df["home_ownership_status"].isnull().sum())
# df["home_ownership_status"].value_counts()

new_df["home_ownership_status"] = df["home_ownership_status"]
new_df.loc[df["home_ownership_status"] == "ANY", "home_ownership_status"] = "OTHER"
new_df.loc[df["home_ownership_status"] == "NONE", "home_ownership_status"] = "OTHER"
new_df.loc[new_df["home_ownership_status"] == "OTHER", "home_ownership_status"] = "OWN"

new_df["home_ownership_status"].value_counts()

MORTGAGE    170710
RENT        137405
OWN          37195
Name: home_ownership_status, dtype: int64

In [358]:
# print(df["delinquency_last_2yrs"].nunique())
# print(df["delinquency_last_2yrs"].isnull().sum())
# df["delinquency_last_2yrs"].value_counts()

# new_df["delinquency_last_2yrs"] = pd.Series()
# new_df.loc[df["delinquency_last_2yrs"] == 0, "delinquency_last_2yrs"] = "==0"
# new_df.loc[df["delinquency_last_2yrs"] == 1, "delinquency_last_2yrs"] = "==1"
# new_df.loc[df["delinquency_last_2yrs"] >= 2, "delinquency_last_2yrs"] = ">=2"

# new_df["delinquency_last_2yrs"].value_counts()

new_df["delinquency_last_2yrs"] = df["delinquency_last_2yrs"]
if val == "train":
    scaler3 = StandardScaler()
    new_df["delinquency_last_2yrs"] = scaler3.fit_transform(df["delinquency_last_2yrs"].values.reshape(-1, 1))
else:
    new_df["delinquency_last_2yrs"] = scaler3.transform(df["delinquency_last_2yrs"].values.reshape(-1, 1))

In [359]:
# print(df["fico_score_range_low"].nunique())
# print(df["fico_score_range_low"].isnull().sum())
# df["fico_score_range_low"].value_counts()

new_df["fico_score_range_low"] = df["fico_score_range_low"]
if val == "train":
    scaler4 = StandardScaler()
    new_df["fico_score_range_low"] = scaler4.fit_transform(df["fico_score_range_low"].values.reshape(-1, 1))
else:
    new_df["fico_score_range_low"] = scaler4.transform(df["fico_score_range_low"].values.reshape(-1, 1))

In [360]:
# print(df["fico_score_range_low"].nunique())
# print(df["fico_score_range_low"].isnull().sum())
# df["fico_score_range_low"].value_counts()

In [361]:
# print(df["fico_score_range_high"].nunique())
# print(df["fico_score_range_high"].isnull().sum())
# df["fico_score_range_high"].value_counts()

# # new_df["fico_score_range_high"] = df["fico_score_range_high"]
# # if val == "train":
# #     scaler5 = StandardScaler()
# #     new_df["fico_score_range_high"] = scaler5.fit_transform(df["fico_score_range_high"].values.reshape(-1, 1))
# # else:
#     new_df["fico_score_range_high"] = scaler5.transform(df["fico_score_range_high"].values.reshape(-1, 1))

In [362]:
# print(df["fico_inquired_last_6mths"].nunique())
# print(df["fico_inquired_last_6mths"].isnull().sum())
# df["fico_inquired_last_6mths"].value_counts()

new_df["fico_inquired_last_6mths"] = df["fico_inquired_last_6mths"]
new_df['fico_inquired_last_6mths'] = new_df["fico_inquired_last_6mths"].fillna(0)

In [363]:
new_df.head()

,requested_amnt,employment_length,type_of_application,loan_duration,state,reason_for_loan,annual_income,debt_to_income_ratio,employment_verified,public_bankruptcies,home_ownership_status,delinquency_last_2yrs,fico_score_range_low,fico_inquired_last_6mths
0,>_5000,< 1 year,Individual,36 months,IN,debt_consolidation,-0.655125,0.065807,Verified,==0,RENT,-0.361806,-0.297979,0.0
1,>_5000,6 years,Individual,36 months,CA,debt_consolidation,-0.591423,1.168144,Source Verified,==0,RENT,-0.361806,-1.194332,0.0
2,>_20000,10+ years,Individual,36 months,FL,other,-0.442785,-0.459799,Verified,==1,MORTGAGE,-0.361806,-1.194332,0.0
3,>_2500,2 years,Individual,36 months,AZ,credit_card,-0.655125,-1.034737,Not Verified,==0,RENT,-0.361806,-1.194332,1.0
4,>_2500,< 1 year,Individual,36 months,CA,other,-0.541877,-1.568416,Source Verified,==0,OWN,-0.361806,-1.194332,0.0


In [364]:
# print(df["revolving_balance"].nunique())
# print(df["revolving_balance"].isnull().sum())
# df[df["revolving_balance"] > 5_000]["revolving_balance"].count()

# new_df["revolving_balance"] = pd.Series()
# new_df.loc[df["revolving_balance"] <= 2_500, "revolving_balance"] = "<_2500"
# new_df.loc[df["revolving_balance"] > 2_500, "revolving_balance"] = ">_2500"
# new_df.loc[df["revolving_balance"] > 5_000, "revolving_balance"] = ">_5000"
# new_df.loc[df["revolving_balance"] > 10_000, "revolving_balance"] = ">_10000"
# new_df.loc[df["revolving_balance"] > 20_000, "revolving_balance"] = ">_20000"

# new_df["revolving_balance"].value_counts()

new_df["revolving_balance"] = df["revolving_balance"]
if val == "train":
    scaler6 = StandardScaler()
    new_df["revolving_balance"] = scaler6.fit_transform(df["revolving_balance"].values.reshape(-1, 1))
else:
    new_df["revolving_balance"] = scaler6.transform(df["revolving_balance"].values.reshape(-1, 1))

In [365]:
# any tax liens

# print(df["any_tax_liens"].nunique())
# print(df["any_tax_liens"].isnull().sum())
# df[df["any_tax_liens"]> 0]["any_tax_liens"].count()

new_df["any_tax_liens"] = df["any_tax_liens"]
new_df.loc[new_df["any_tax_liens"] > 0, "any_tax_liens"] = 1
new_df['any_tax_liens'] = new_df["any_tax_liens"].fillna(0)

In [366]:
# new_df.to_csv("parsed_df.csv")

In [367]:
new_df.shape

(345310, 16)

In [368]:
y_label = df["loan_paid"]
one_hot_encode = new_df.loc[:, new_df.dtypes == 'object']
non_one_hot_encode = new_df.loc[:, new_df.dtypes != 'object']

one_hot_encode = pd.get_dummies(one_hot_encode, drop_first=False)
one_hot_encode

,requested_amnt_<_2500,requested_amnt_>_10000,requested_amnt_>_20000,requested_amnt_>_2500,requested_amnt_>_30000,requested_amnt_>_5000,employment_length_1 year,employment_length_10+ years,employment_length_2 years,employment_length_3 years,...,reason_for_loan_other,employment_verified_Not Verified,employment_verified_Source Verified,employment_verified_Verified,public_bankruptcies_==0,public_bankruptcies_==1,public_bankruptcies_>=2,home_ownership_status_MORTGAGE,home_ownership_status_OWN,home_ownership_status_RENT
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
2,0,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,1,0,1,0,0
3,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345305,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
345306,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
345307,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,1,0,0
345308,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1


In [369]:
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

# svd = TruncatedSVD(n_components=10 , random_state=42)
# tsne = TSNE(random_state=42)


# one_hot_encode = pd.DataFrame(svd.fit_transform(one_hot_encode))

In [370]:
final_df = pd.concat([one_hot_encode, non_one_hot_encode], ignore_index=True, sort=False, axis=1)
final_df.columns = np.concatenate([one_hot_encode.columns, non_one_hot_encode.columns])
final_df

,requested_amnt_<_2500,requested_amnt_>_10000,requested_amnt_>_20000,requested_amnt_>_2500,requested_amnt_>_30000,requested_amnt_>_5000,employment_length_1 year,employment_length_10+ years,employment_length_2 years,employment_length_3 years,...,home_ownership_status_MORTGAGE,home_ownership_status_OWN,home_ownership_status_RENT,annual_income,debt_to_income_ratio,delinquency_last_2yrs,fico_score_range_low,fico_inquired_last_6mths,revolving_balance,any_tax_liens
0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,-0.655125,0.065807,-0.361806,-0.297979,0.0,0.099462,0.0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,1,-0.591423,1.168144,-0.361806,-1.194332,0.0,-0.494430,0.0
2,0,0,1,0,0,0,0,1,0,0,...,1,0,0,-0.442785,-0.459799,-0.361806,-1.194332,0.0,-0.474001,0.0
3,0,0,0,1,0,0,0,0,1,0,...,0,0,1,-0.655125,-1.034737,-0.361806,-1.194332,1.0,-0.549809,0.0
4,0,0,0,1,0,0,0,0,0,0,...,0,1,0,-0.541877,-1.568416,-0.361806,-1.194332,0.0,-0.710661,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345305,0,0,0,1,0,0,0,0,0,0,...,0,1,0,-0.740061,-1.123534,-0.361806,-1.194332,0.0,-0.560600,0.0
345306,0,1,0,0,0,0,0,0,0,0,...,0,1,0,-0.159665,0.092715,0.774223,-1.194332,2.0,-0.277222,0.0
345307,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0.152163,0.935837,1.910253,-0.746156,1.0,-0.371548,0.0
345308,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0.341457,0.264030,-0.361806,0.448982,0.0,0.126863,0.0


In [371]:
final_df.columns

Index(['requested_amnt_<_2500', 'requested_amnt_>_10000',
       'requested_amnt_>_20000', 'requested_amnt_>_2500',
       'requested_amnt_>_30000', 'requested_amnt_>_5000',
       'employment_length_1 year', 'employment_length_10+ years',
       'employment_length_2 years', 'employment_length_3 years',
       'employment_length_4 years', 'employment_length_5 years',
       'employment_length_6 years', 'employment_length_7 years',
       'employment_length_8 years', 'employment_length_9 years',
       'employment_length_< 1 year', 'type_of_application_Individual',
       'type_of_application_Joint App', 'loan_duration_ 36 months',
       'loan_duration_ 60 months', 'state_AK', 'state_AL', 'state_AR',
       'state_AZ', 'state_CA', 'state_CO', 'state_CT', 'state_DC', 'state_DE',
       'state_FL', 'state_GA', 'state_HI', 'state_IA', 'state_ID', 'state_IL',
       'state_IN', 'state_KS', 'state_KY', 'state_LA', 'state_MA', 'state_MD',
       'state_ME', 'state_MI', 'state_MN', 'state_MO'

In [372]:
# from imblearn.over_sampling import RandomOverSampler
# over = RandomOverSampler(random_state=42)

# final_df, y_label = over.fit_resample(final_df, y_label)

In [373]:
if val == "train" and False:
    from sklearn.feature_selection import RFE
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

    rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10, verbose=3)
    rfe.fit(final_df, y_label)

In [374]:
# rfe.ranking_

In [375]:
# chosen_feature = rfe.get_feature_names_out(np.concatenate([one_hot_encode.columns, non_one_hot_encode.columns]))
# chosen_feature

In [376]:
# final_df.columns = np.concatenate([one_hot_encode.columns, non_one_hot_encode.columns])
# final_df.loc[:, chosen_feature]

In [377]:
# final_df = final_df.loc[:, chosen_feature]

In [378]:
if val == "test":
    y_pred = clf8.predict(final_df)
    # y_pred = mlp_nn.predict(final_df)
    final_pred_df = pd.concat([df["ID"], pd.Series(y_pred)], ignore_index=True, sort=False, axis=1)
    final_pred_df.columns = ["ID", "loan_paid"]
    final_pred_df.head()
    final_pred_df.set_index('ID', inplace=True)
    final_pred_df.to_csv("lending_pred_mlp_60_new_fea.csv")

c:\Users\fongm\Anaconda3\envs\ixperience\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearDiscriminantAnalysis was fitted without feature names
  warnings.warn(


In [379]:
if val == "test": raise

RuntimeError: No active exception to reraise

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# under = RandomUnderSampler(sampling_strategy={1: 0.5, 0: 0.5})
# under = RandomUnderSampler(random_state=42)
over = RandomOverSampler(random_state=42)

final_df, y_label = over.fit_resample(final_df, y_label)
# y_label.value_counts()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import  precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,auc, roc_curve, plot_confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


clf1 = RandomForestClassifier(random_state=42)
clf2 = SVC(probability=True, random_state=42)
clf4 = DecisionTreeClassifier(random_state=42)
clf5 = KNeighborsClassifier()
clf6 = MultinomialNB()
clf7 = GradientBoostingClassifier(random_state=42)
clf8 = LDA()
mlp_nn = MLPClassifier(solver = 'adam', hidden_layer_sizes=(60,), early_stopping=True, learning_rate_init=0.0007 , random_state = 42, max_iter = 1000, verbose = 3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_df.values, y_label.values.ravel(), test_size=0.2, random_state=42, stratify=y_label)

In [ ]:
# from autosklearn.classification import AutoSklearnClassifier

# model = AutoSklearnClassifier(time_left_for_this_task=2*60, per_run_time_limit=30, n_jobs=8)
# model.fit(X_train, y_train)
# # summarize
# print(model.sprint_statistics())
# # evaluate best model
# y_pred = model.predict(X_test)
# print(classification_report(y_pred, y_test))


In [ ]:
# parameters = {'hidden_layer_sizes': [(20,), (20,10), (20, 10, 2)], 'learning_rate_init':[0.0001, 0.001, 0.01, 0.1]}
# clf_nn = GridSearchCV(mlp_nn, parameters, cv = 5, verbose=3).fit(X_train, y_train)

mlp_nn.fit(X_train, y_train)
y_pred = mlp_nn.predict(X_test)

print(classification_report(y_pred, y_test))
fpr, tpr, thresholds = roc_curve(y_test,y_pred)
roc_auc = auc(fpr, tpr)
print(roc_auc)


Iteration 1, loss = 0.63415226
Validation score: 0.641764
Iteration 2, loss = 0.62962556
Validation score: 0.643132
Iteration 3, loss = 0.62808222
Validation score: 0.644991
Iteration 4, loss = 0.62714246
Validation score: 0.646484
Iteration 5, loss = 0.62653275
Validation score: 0.646085
Iteration 6, loss = 0.62608160
Validation score: 0.645366
Iteration 7, loss = 0.62569460
Validation score: 0.646695
Iteration 8, loss = 0.62542801
Validation score: 0.646515
Iteration 9, loss = 0.62516684
Validation score: 0.647492
Iteration 10, loss = 0.62493880
Validation score: 0.647742
Iteration 11, loss = 0.62476463
Validation score: 0.647414
Iteration 12, loss = 0.62454402
Validation score: 0.647992
Iteration 13, loss = 0.62444441
Validation score: 0.646445
Iteration 14, loss = 0.62426971
Validation score: 0.648656
Iteration 15, loss = 0.62415641
Validation score: 0.646492
Iteration 16, loss = 0.62401208
Validation score: 0.648594
Iteration 17, loss = 0.62391894
Validation score: 0.647625
Iterat

In [ ]:
# clf7.fit(X_train, y_train)
# y_pred = clf7.predict(X_test)

# from sklearn.metrics import classification_report
# print(classification_report(y_pred, y_test))

In [ ]:
clf8.fit(X_train, y_train)
y_pred = clf8.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

fpr, tpr, thresholds = roc_curve(y_test,y_pred)
roc_auc = auc(fpr, tpr)
print(roc_auc)

              precision    recall  f1-score   support

           0       0.61      0.64      0.62    153655
           1       0.65      0.63      0.64    166293

    accuracy                           0.63    319948
   macro avg       0.63      0.63      0.63    319948
weighted avg       0.63      0.63      0.63    319948

0.6317995424256443


In [ ]:
raise

RuntimeError: No active exception to reraise

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf8, X_train, y_train, cv=5, scoring='precision')
print(scores)

KeyboardInterrupt: 

In [ ]:

# param7 = {}
# # param7['loss'] = ["log_loss", "deviance", "exponential"]
# param7['n_estimators'] = [50, 100, 250]
# param7['max_depth'] = [10, 20]
   
# grid = GridSearchCV(clf7, param7, scoring="precision", refit = True, verbose = 3,n_jobs=-1) 
   
# # fitting the model for grid search 
# grid.fit(X_train, y_train)
# print(grid.best_params_) 
# grid_predictions = grid.predict(X_test) 

# # print classification report 
# print(classification_report(y_test, grid_predictions))

In [ ]:
raise

RuntimeError: No active exception to reraise

In [ ]:

clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.59      0.57      0.58     41584
           1       0.55      0.58      0.57     38468

    accuracy                           0.57     80052
   macro avg       0.57      0.57      0.57     80052
weighted avg       0.58      0.57      0.57     80052



In [ ]:
# clf2.fit(X_train, y_train)
# y_pred = clf1.predict(X_test)

# from sklearn.metrics import classification_report
# print(classification_report(y_pred, y_test))

In [ ]:
# clf4.fit(X_train, y_train)
# y_pred = clf4.predict(X_test)

# from sklearn.metrics import classification_report
# print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.54      0.54      0.54     40081
           1       0.54      0.54      0.54     39971

    accuracy                           0.54     80052
   macro avg       0.54      0.54      0.54     80052
weighted avg       0.54      0.54      0.54     80052



In [ ]:
clf5.fit(X_train, y_train)
y_pred = clf5.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.57      0.56      0.57     40282
           1       0.56      0.57      0.56     39770

    accuracy                           0.56     80052
   macro avg       0.56      0.56      0.56     80052
weighted avg       0.56      0.56      0.56     80052



In [ ]:
# clf6.fit(X_train, y_train)
# y_pred = clf6.predict(X_test)

# from sklearn.metrics import classification_report
# print(classification_report(y_pred, y_test))

In [ ]:
clf8.fit(X_train, y_train)
y_pred = clf8.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.63      0.60      0.62     42075
           1       0.58      0.61      0.60     37977

    accuracy                           0.61     80052
   macro avg       0.61      0.61      0.61     80052
weighted avg       0.61      0.61      0.61     80052



In [ ]:
clf7.fit(X_train, y_train)
y_pred = clf7.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.64      0.61      0.62     42440
           1       0.58      0.62      0.60     37612

    accuracy                           0.61     80052
   macro avg       0.61      0.61      0.61     80052
weighted avg       0.61      0.61      0.61     80052



In [ ]:
raise

RuntimeError: No active exception to reraise

In [ ]:
param1 = {}
param1['classifier__n_estimators'] = [10, 50, 100, 250]
param1['classifier__max_depth'] = [5, 10, 20]
param1['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param1['classifier'] = [clf1]

param2 = {}
param2['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param2['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param2['classifier'] = [clf2]

param4 = {}
param4['classifier__max_depth'] = [5,10,25,None]
param4['classifier__min_samples_split'] = [2,5,10]
param4['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param4['classifier'] = [clf4]

param5 = {}
param5['classifier__n_neighbors'] = [2,5,10,25,50]
param5['classifier'] = [clf5]

param6 = {}
param6['classifier__alpha'] = [10**0, 10**1, 10**2]
param6['classifier'] = [clf6]

param7 = {}
param7['classifier__n_estimators'] = [10, 50, 100, 250]
param7['classifier__max_depth'] = [5, 10, 20]
param7['classifier'] = [clf7]

param8 = {}
param8["classifier__shrinkage"] = np.arange(0.001, 0.5, 0.001)
param8["classifier__solver"] = ["eigen"]
param8['classifier'] = [clf8]

In [ ]:
final_df.isna().sum()

requested_amnt_<_2500                    0
requested_amnt_>_10000                   0
requested_amnt_>_20000                   0
requested_amnt_>_2500                    0
requested_amnt_>_30000                   0
requested_amnt_>_5000                    0
requested_amnt_other                     0
employment_length_0-2_years              0
employment_length_10+ years              0
employment_length_3-5_years              0
employment_length_6-9_years              0
employment_length_other                  0
reason_for_loan_credit_card              0
reason_for_loan_debt_consolidation       0
reason_for_loan_other                    0
annual_income_<_25000                    0
annual_income_>_100000                   0
annual_income_>_25000                    0
annual_income_>_50000                    0
annual_income_>_75000                    0
employment_verified_Not Verified         0
employment_verified_Source Verified      0
employment_verified_Verified             0
public_bank

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_df, y_label.values.ravel(), test_size=0.2, random_state=42, stratify=y_label)
pipeline = Pipeline([('classifier', clf1)])
params = [param1, param2, param4, param5, param6, param7, param8]
gs = GridSearchCV(pipeline, params, cv=10, scoring='accuracy').fit(X_train, y_train)

c:\Users\fongm\Anaconda3\envs\m2vec\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
6350 fits failed out of a total of 6350.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
480 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\fongm\Anaconda3\envs\m2vec\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\fongm\Anaconda3\envs\m2vec\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\fongm\Anaconda3\envs\m2vec\lib\site-packages\sklearn\ensemble\_forest.py", line 328, in fit
    X, y, mu

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
print(gs.best_params_)
print(gs.best_score_)